In [1]:
import torch
import attr
from pathlib import Path
from torchvision import datasets, transforms
import torchvision
import matplotlib.pyplot as plt
from sklearn import metrics as sk_metrics
import numpy as np

In [3]:
@attr.s(auto_attribs=True)
class DataPreparation:
    data_dir: Path
    device: str = attr.ib(default=torch.device("cuda:0" if torch.cuda.is_available() else "cpu"), init=False)

    @staticmethod
    def data_transformations():
        data_transforms = {
            'test': transforms.Compose([
                transforms.CenterCrop((200,1500)),
                transforms.Resize((224,224)),
                transforms.ToTensor(),
                transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
            ])
        }
        return data_transforms
    
    def create_dataloaders(self, batch_size, shuffle, num_workers):
        data_transforms = self.data_transformations()
        
        image_datasets = {
            'test': datasets.ImageFolder(self.data_dir, data_transforms['test'])
        }
        dataloaders = {
            'test': torch.utils.data.DataLoader(image_datasets['test'], batch_size=batch_size, shuffle=shuffle, num_workers=num_workers)
        }
        dataset_sizes = {
            'test': len(image_datasets['test'])
        }
        return dataloaders, dataset_sizes

In [4]:
data_prep = DataPreparation('../data/figures/test')
data, size = data_prep.create_dataloaders(16, False, 4)

In [15]:
model_path = Path().cwd().parents[0] / "models/resnet50_d_02_t_20_28_initial_beat.pth" 
model = torch.load(model_path)
model = model.eval()

In [16]:
y_pred = []
y_true = []

for inputs, labels in data['test']:
    inputs = inputs.to(0)
    labels = labels.to(0)
    outputs = model(inputs)
    _, preds = torch.max(outputs, 1)
    y_pred.append(preds.cpu().numpy())
    y_true.append(labels.cpu().numpy())


In [17]:
y_pred = np.concatenate(y_pred)
y_true = np.concatenate(y_true)

In [18]:
acc = sk_metrics.accuracy_score(y_pred, y_true)
f1 = sk_metrics.f1_score(y_pred, y_true)
pre = sk_metrics.precision_score(y_pred, y_true)

In [19]:
acc, f1, pre

(0.9828452279404796, 0.9898616479023132, 0.9909162274307503)